In [ ]:
def extend_embed(embed_dict, vocab_size):
    all_embs = np.stack(embed_dict.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    nb_words = min(vocab_size, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= vocab_size: continue
        embedding_vector = embed_dict.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix= np.concatenate((glove_embed_matrix, wiki_embed_matrix), axis=1)
embedding_matrix= np.mean((glove_embed_matrix, wiki_embed_matrix, google_embed_matrix), axis=0)


In [ ]:
## Indirect features

#Sentense count in each comment:
def add_meta_features(df):
    # '\n' can be used to count the number of sentences in each comment
    df['count_sent']=df["comment_text"].apply(lambda x: len(re.findall("\n",str(x)))+1)
    #Word count in each comment:
    df['count_word']=df["comment_text"].apply(lambda x: len(str(x).split()))
    #Unique word count
    df['count_unique_word']=df["comment_text"].apply(lambda x: len(set(str(x).split())))
    #Letter count
    df['count_letters']=df["comment_text"].apply(lambda x: len(str(x)))
    #punctuation count
    df["count_punctuations"] =df["comment_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    #upper case words count
    df["count_words_upper"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    #title case words count
    df["count_words_title"] = df["comment_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
    #Number of stopwords
    df["count_stopwords"] = df["comment_text"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
    #Average length of the words
    df["mean_word_len"] = df["comment_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    return df


In [ ]:
train_meta_feat= train.iloc[:,9:].values
test_meta_feat= test.iloc[:,3:].values
n_meta_feat= train_meta_feat.shape[1]

In [ ]:
embed_size= embedding_matrix.shape[1]
inp = Input(shape=(maxlen,))
# x = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable= False)(inp)
x = Embedding(vocab_size, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
meta_inp= Input(shape= (n_meta_feat,))
x= concatenate([x, meta_inp])
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=[inp, meta_inp], outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
embed_size= embedding_matrix.shape[1]
inp = Input(shape=(maxlen,))
x = Embedding(vocab_size, embed_size, weights=[embedding_matrix])(inp)
x = SpatialDropout1D(0.3)(x)
x = Bidirectional(GRU(128, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
meta_inp= Input(shape= (n_meta_feat,))
x = concatenate([avg_pool, max_pool, meta_inp])
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=[inp, meta_inp], outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])